## CIFAR 10

In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
from fastai.conv_learner import *
PATH = Path("data/cifar/")
os.makedirs(PATH,exist_ok=True)
torch.cuda.set_device(0)

In [9]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

num_workers = num_cpus()//2
bs=256
sz=32
df = pd.read_csv(PATH/'train_full.csv')
val_idx = get_cv_idxs(len(df))

In [10]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
data = ImageClassifierData.from_csv(PATH,'train', PATH/'train_full.csv',val_idxs = val_idx,  test_name='test', tfms=tfms, bs=bs)

In [11]:
def conv_layer(ni, nf, ks=3, stride=1):
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=ks, bias=False, stride=stride, padding=ks//2),
        nn.BatchNorm2d(nf, momentum=0.01),
        nn.LeakyReLU(negative_slope=0.1, inplace=True))

In [12]:
class ResLayer(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.conv1=conv_layer(ni, ni//2, ks=1)
        self.conv2=conv_layer(ni//2, ni, ks=3)
        
    def forward(self, x): return x.add(self.conv2(self.conv1(x)))

In [13]:
class Darknet(nn.Module):
    def make_group_layer(self, ch_in, num_blocks, stride=1):
        return [conv_layer(ch_in, ch_in*2,stride=stride)
               ] + [(ResLayer(ch_in*2)) for i in range(num_blocks)]

    def __init__(self, num_blocks, num_classes, nf=32):
        super().__init__()
        layers = [conv_layer(3, nf, ks=3, stride=1)]
        for i,nb in enumerate(num_blocks):
            layers += self.make_group_layer(nf, nb, stride=2-(i==1))
            nf *= 2
        layers += [nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(nf, num_classes)]
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x): return self.layers(x)

In [14]:
m = Darknet([1, 2, 4, 6, 3], num_classes=10, nf=32)


In [15]:
learn = ConvLearner.from_model_data(to_gpu(m), data)
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]
learn.clip = 1e-1
%time learn.fit(1.5, 1, wds=1e-4, cycle_len=20, use_clr_beta=(12, 15, 0.95, 0.85))

epoch      trn_loss   val_loss   accuracy                   
    0      1.524506   2.474388   0.0992    
    1      1.243316   2.90524    0.0995                     
    2      1.03968    2.102872   0.3579                     
    3      0.917148   1.593577   0.4531                      
    4      0.824389   1.191766   0.5781                      
    5      0.746274   1.820095   0.4044                      
    6      0.701335   1.558274   0.4546                      
    7      0.663048   1.429027   0.5166                      
    8      0.640388   1.550357   0.4525                      
    9      0.589545   1.474113   0.4623                      
    10     0.554648   1.593462   0.4637                      
    11     0.536154   1.369861   0.5005                      
    12     0.509677   1.121718   0.6044                      
    13     0.468491   1.173644   0.6029                      
    14     0.438638   1.009462   0.668                       
    15     0.385817   0.67463

[array([0.28759]), 0.9016]

In [16]:
learn = ConvLearner.from_model_data(to_gpu(m), data)
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]
learn.clip = 1e-1
%time learn.fit(1.5, 1, wds=1e-4, cycle_len=30, use_clr_beta=(12, 15, 0.95, 0.85))

epoch      trn_loss   val_loss   accuracy                    
    0      0.281255   0.458668   0.8495    
    1      0.326356   0.746909   0.7394                      
    2      0.375776   0.777425   0.7492                      
    3      0.393191   0.930965   0.6943                      
    4      0.42376    0.919787   0.698                       
    5      0.439042   0.775543   0.7327                      
    6      0.451626   1.323106   0.6207                      
    7      0.47254    0.855716   0.7257                      
    8      0.468085   1.514426   0.5597                      
    9      0.478741   2.049475   0.4939                      
    10     0.464222   0.89003    0.7167                      
    11     0.475184   1.049708   0.6676                      
    12     0.464974   0.986994   0.694                       
    13     0.449753   0.854673   0.7103                      
    14     0.45223    0.919351   0.7092                      
    15     0.42309    0.98

[array([0.22373]), 0.9284]

In [29]:
df2 = pd.read_csv(PATH/'train_10.csv')
val_idx2 = get_cv_idxs(len(df2))
df3 = pd.read_csv(PATH/'train_100.csv')
val_idx3 = get_cv_idxs(len(df3))
df4 = pd.read_csv(PATH/'train_1000.csv')
val_idx4 = get_cv_idxs(len(df4))
df5 = pd.read_csv(PATH/'train_2000.csv')
val_idx5 = get_cv_idxs(len(df5))
df6 = pd.read_csv(PATH/'train_3000.csv')
val_idx6 = get_cv_idxs(len(df6))


In [30]:
sz = 32
bs = 128
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
data = ImageClassifierData.from_csv(PATH,'train', PATH/'train_full.csv',val_idxs = val_idx,  test_name='test', tfms=tfms, bs=bs)
data2 = ImageClassifierData.from_csv(PATH,'train', PATH/'train_10.csv',val_idxs = val_idx2,  test_name='test', tfms=tfms, bs=bs)
data3 = ImageClassifierData.from_csv(PATH,'train', PATH/'train_100.csv',val_idxs = val_idx3,  test_name='test', tfms=tfms, bs=bs)
data4 = ImageClassifierData.from_csv(PATH,'train', PATH/'train_1000.csv',val_idxs = val_idx4,  test_name='test', tfms=tfms, bs=bs)
data5 = ImageClassifierData.from_csv(PATH,'train', PATH/'train_2000.csv',val_idxs = val_idx5,  test_name='test', tfms=tfms, bs=bs)
data6 = ImageClassifierData.from_csv(PATH,'train', PATH/'train_3000.csv',val_idxs = val_idx6,  test_name='test', tfms=tfms, bs=bs)

In [31]:
learn = ConvLearner.from_model_data(to_gpu(m), data2)
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]
learn.clip = 1e-1

In [32]:
def phases_1cycle(cycle_len,lr,div,pct,max_mom,min_mom):
    tri_cyc = (1-pct/100) * cycle_len
    return [TrainingPhase(epochs=tri_cyc/2, opt_fn=optim.SGD, lr=(lr/div,lr), lr_decay=DecayType.LINEAR, 
                          momentum=(max_mom,min_mom), momentum_decay=DecayType.LINEAR),
           TrainingPhase(epochs=tri_cyc/2, opt_fn=optim.SGD, lr=(lr,lr/div), lr_decay=DecayType.LINEAR, 
                          momentum=(min_mom,max_mom), momentum_decay=DecayType.LINEAR),
           TrainingPhase(epochs=cycle_len-tri_cyc, opt_fn=optim.SGD, lr=(lr/div,lr/(100*div)), lr_decay=DecayType.LINEAR, 
                          momentum=max_mom)]

In [33]:

%time learn.fit_opt_sched(phases_1cycle(20, 1.5, 12, 15, 0.95, 0.85), data_list = [data2,data3,data])

epoch      trn_loss   val_loss   accuracy                 
    0      0.077837   0.201433   0.954545  
    1      0.065972   0.204647   0.954545                
    2      0.066401   0.206578   0.954545                 
    3      0.05331    0.216111   0.954545                 
    4      0.044311   0.296352   0.909091                 
    5      0.03775    0.45941    0.863636                 
    6      0.032981   0.521618   0.863636                
    7      0.029713   0.657549   0.772727                 
    8      0.027217   0.773334   0.772727         
    9      0.40318    3.415553   0.390756                 
    10     0.456303   7.958943   0.231092                
    11     0.461087   3.300434   0.5                     
    12     0.436064   4.782289   0.403361                
    13     0.40541    2.283725   0.52521                 
    14     0.377065   3.656712   0.457983                
    15     0.343089   1.848455   0.609244                
    16     0.300592   1.7747

[array([0.25286]), 0.9164]

In [34]:
learn2 = ConvLearner.from_model_data(to_gpu(m), data2)
learn2.crit = nn.CrossEntropyLoss()
learn2.metrics = [accuracy]
learn2.clip = 1e-1
%time learn2.fit_opt_sched(phases_1cycle(20, 1.5, 12, 15, 0.95, 0.85), data_list = [data2,data3,data4,data])

epoch      trn_loss   val_loss   accuracy                 
    0      0.020608   0.089935   0.954545  
    1      0.020659   0.084502   0.954545                 
    2      0.019161   0.078669   1.0                      
    3      0.016231   0.08818    1.0                      
    4      0.013758   0.138457   0.909091                 
    5      0.011888   0.274588   0.909091                 
    6      0.01036    0.536643   0.818182                 
    7      0.009293   0.97149    0.772727                  
    8      0.008409   1.773561   0.681818          
    9      0.268992   4.241541   0.352941                 
    10     0.306918   5.961612   0.289916                
    11     0.310805   3.656671   0.365546                
    12     0.30296    2.33097    0.491597                
    13     0.285912   2.368432   0.504202                
    14     0.267028   6.106249   0.252101                
    15     0.244499   2.540092   0.5                     
    16     0.214331   2.

[array([0.21732]), 0.9241034195659755]

In [35]:
learn3 = ConvLearner.from_model_data(to_gpu(m), data2)
learn3.crit = nn.CrossEntropyLoss()
learn3.metrics = [accuracy]
learn3.clip = 1e-1
%time learn3.fit_opt_sched(phases_1cycle(20, 1.5, 12, 15, 0.95, 0.85), data_list = [data2,data3,data4,data5,data6])

epoch      trn_loss   val_loss   accuracy                 
    0      0.012071   0.14609    0.909091  
    1      0.012682   0.149518   0.909091                 
    2      0.011746   0.161856   0.909091                 
    3      0.010722   0.195552   0.909091                 
    4      0.009187   0.339415   0.863636                  
    5      0.008327   0.576228   0.863636                  
    6      0.007437   0.591503   0.727273                  
    7      0.007282   1.11577    0.636364                  
    8      0.00668    1.830211   0.545455          
    9      0.158858   1.4402     0.647059                 
    10     0.212127   1.411529   0.672269                
    11     0.227161   2.089364   0.537815                
    12     0.215724   2.321228   0.563025                
    13     0.208561   2.399389   0.55042                 
    14     0.198698   1.233888   0.705882                
    15     0.184769   1.287521   0.718487                
    16     0.162767  

[array([0.27712]), 0.9028356966125259]

In [37]:
learn4 = ConvLearner.from_model_data(to_gpu(m), data2)
learn4.crit = nn.CrossEntropyLoss()
learn4.metrics = [accuracy]
learn4.clip = 1e-1
%time learn4.fit_opt_sched(phases_1cycle(20, 1.5, 12, 15, 0.95, 0.85), data_list = [data2,data3,data4])

epoch      trn_loss   val_loss   accuracy                  
    0      0.008756   0.21496    0.954545  
    1      0.009497   0.219655   0.954545                 
    2      0.008013   0.225424   0.954545                  
    3      0.006806   0.223637   0.909091                  
    4      0.005961   0.198287   0.954545                  
    5      0.005216   0.195376   0.954545                  
    6      0.004627   0.248606   0.909091                  
    7      0.004142   0.434913   0.909091                  
    8      0.003752   0.688438   0.818182          
    9      0.093153   1.689604   0.621849                 
    10     0.119101   3.131015   0.512605                 
    11     0.149679   2.016418   0.642857                
    12     0.154705   1.723721   0.592437                
    13     0.161438   2.671733   0.579832                
    14     0.154604   2.558146   0.596639                
    15     0.145384   2.998436   0.546218                
    16     0.1357

[array([0.2551]), 0.9132610512237135]

In [38]:
def get_TTA_accuracy(learn):
    preds, targs = learn.TTA()
    # combining the predictions across augmented and non augmented inputs
    preds = 0.6 * preds[0] + 0.4 * preds[1:].sum(0)
    return accuracy_np(preds, targs)

In [39]:
get_TTA_accuracy(learn)

0.9545454545454546

In [40]:
get_TTA_accuracy(learn2)

0.9545454545454546

In [41]:
get_TTA_accuracy(learn3)

0.9545454545454546

In [43]:
get_TTA_accuracy(learn4)

0.9545454545454546

In [46]:
learn = ConvLearner.from_model_data(to_gpu(m), data2)
learn.crit = nn.CrossEntropyLoss()
learn.metrics = [accuracy]
learn.clip = 1e-1
%time learn.fit_opt_sched(phases_1cycle(20, 1.5, 12, 15, 0.95, 0.85), data_list = [data2,data3,data])

epoch      trn_loss   val_loss   accuracy                
    0      0.114095   1.253506   0.681818  
    1      0.1288     1.232286   0.681818                
    2      0.124818   1.188657   0.681818                
    3      0.113357   1.132311   0.681818                
    4      0.100078   1.067754   0.681818              
    5      0.086392   1.017213   0.681818                 
    6      0.076283   0.981587   0.727273                 
    7      0.067104   0.985963   0.727273                 
    8      0.061454   1.035018   0.636364         
    9      0.233301   0.821028   0.773109                 
    10     0.22115    0.703577   0.802521                
    11     0.200859   0.785548   0.785714                
    12     0.16965    0.782593   0.802521                
    13     0.14534    0.895026   0.768908                
    14     0.124626   0.548713   0.831933                
    15     0.110902   0.472831   0.836134                
    16     0.096566   0.505455   

[array([0.23725]), 0.9239]

In [48]:
get_TTA_accuracy(learn)

0.9545454545454546

In [49]:
learn2 = ConvLearner.from_model_data(to_gpu(m), data2)
learn2.crit = nn.CrossEntropyLoss()
learn2.metrics = [accuracy]
learn2.clip = 1e-1
%time learn2.fit_opt_sched(phases_1cycle(20, 1.5, 12, 15, 0.95, 0.85), data_list = [data2,data3,data4,data])

epoch      trn_loss   val_loss   accuracy                  
    0      0.00301    0.072421   0.954545  
    1      0.004309   0.071899   0.954545                  
    2      0.004292   0.069052   0.954545                  
    3      0.003547   0.064639   0.954545                  
    4      0.003092   0.060678   0.954545                  
    5      0.002819   0.059256   0.954545                  
    6      0.002501   0.058061   0.954545                 
    7      0.002348   0.060819   0.954545                  
    8      0.002171   0.068793   0.954545          
    9      0.029927   0.249701   0.928571                  
    10     0.033921   0.345849   0.886555                 
    11     0.040334   0.304073   0.903361                 
    12     0.043829   0.37094    0.882353                 
    13     0.044027   0.799603   0.785714                 
    14     0.040961   0.522748   0.861345                 
    15     0.037586   0.39877    0.87395                  
    16     

[array([0.13888]), 0.9553794831012665]

In [53]:
get_TTA_accuracy(learn2)

1.0

In [52]:
learn4 = ConvLearner.from_model_data(to_gpu(m), data2)
learn4.crit = nn.CrossEntropyLoss()
learn4.metrics = [accuracy]
learn4.clip = 1e-1
%time learn4.fit_opt_sched(phases_1cycle(20, 1.5, 12, 15, 0.95, 0.85), data_list = [data2,data3,data4])

epoch      trn_loss   val_loss   accuracy                   
    0      0.000662   0.002235   1.0       
    1      0.001481   0.002218   1.0                       
    2      0.001414   0.002178   1.0                       
    3      0.001326   0.002043   1.0                       
    4      0.001127   0.001908   1.0                       
    5      0.001004   0.001813   1.0                     
    6      0.000976   0.001745   1.0                        
    7      0.000907   0.001684   1.0                        
    8      0.000827   0.001692   1.0                
    9      0.008388   0.208951   0.936975                  
    10     0.013545   0.267088   0.911765                  
    11     0.015429   0.569955   0.852941                 
    12     0.020097   0.918275   0.785714                 
    13     0.026946   0.670365   0.848739                 
    14     0.026716   0.491037   0.869748                 
    15     0.025755   0.333822   0.890756                 
    16 

[array([0.16409]), 0.9424520434688984]

In [54]:
get_TTA_accuracy(learn4)

1.0